# NBA Lineup Prediction - Machine Learning Project
#### Fahad Hussain, Rajiv Lomada, and Kevaun Harris (Group 21)

Left to do:
- explain why no engineered specific features
- make sure final code runs
- fix top 3 mentions to top 2
- review full document and ensure it explains everything (like why the model chooses the players it does)
- mention what years it is trained on, and what years it is tested on.

## Stage 1: Prepare Data
Removed all columns from the CSV files that were not allowed as features to ensure efficiency and proper model training. Modified the 2015 CSV data to name Charlotte Hornets as "CHA" rather than "CHO", ensuring consistency with training data. 

## Stage 2: Load and Process Data
### `load_data(directory)`
Loads and combines all the CSV files (2007-2015) into a single dataframe, creating one large dataset for further processing.
### `sort_players(row)`
Sorts the home and away lineups alphabetically to ensure duplicate lineups are removed later. This avoids the model mistakenly learning position-based patterns that don't exist (the player data is not organized position-wise).

In [ ]:
import os
import pandas as pd

def load_data(data_dir):
    all_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
    df_list = []

    for file in all_files:
        df = pd.read_csv(file)
        df_list.append(df)
    df = pd.concat(df_list, ignore_index=True)

    def sort_players(row):
        home_players = sorted([row[f'home_{i}'] for i in range(5)])
        away_players = sorted([row[f'away_{i}'] for i in range(5)])
        for i in range(5):
            row[f'home_{i}'] = home_players[i]
            row[f'away_{i}'] = away_players[i]
        return row

    df = df.apply(sort_players, axis=1)
    return df

### `create_roster(dataframe)`
Creates a dictionary storing team rosters for each team and season.
Example: `(GSW, 2015) = [Steph Curry, Klay Thompson, Draymond Green, ...]`
This ensures that for any given test case, we have a valid list of eligible players.

In [ ]:
def create_roster(df):
    roster = df.groupby(['home_team', 'season'])[[f'home_{i}' for i in range(5)]].apply(
        lambda x: set(pd.unique(x.values.ravel()))
    ).to_dict()
    return roster

### `encode_features(dataframe, encoders)`
Adds numerical encoding for categorical columns (players, teams, seasons) using `LabelEncoder`, allowing the model to process them correctly.

In [ ]:
from sklearn.preprocessing import LabelEncoder

def encode_features(df, player_encoder, team_encoder, season_encoder):
    df['home_team_encoded'] = team_encoder.transform(df['home_team'])
    df['away_team_encoded'] = team_encoder.transform(df['away_team'])
    df['season_encoded'] = season_encoder.transform(df['season'])
    for i in range(5):
        df[f'home_{i}_encoded'] = player_encoder.transform(df[f'home_{i}'])
        df[f'away_{i}_encoded'] = player_encoder.transform(df[f'away_{i}'])
    return df

## Stage 3: Train Model
The encoded columns are used as input features, and the outcome column is used as the target.
We trained A **Random Forest** model because it is well-suited for capturing complex patterns and relationships between players and teams.
The trained model and encoders are saved as `.pkl` files for reuse.

In [ ]:
# Gather input features and target
X = df[['home_team_encoded', 'season_encoded',
       'home_0_encoded', 'home_1_encoded', 'home_2_encoded', 'home_3_encoded', 'home_4_encoded',
       'away_0_encoded', 'away_1_encoded', 'away_2_encoded', 'away_3_encoded', 'away_4_encoded']]
y = df['outcome']

# Train Random Forest model
model = RandomForestClassifier(n_estimators=300, max_depth=None, random_state=1, n_jobs=-1)
model.fit(X, y)

# Save model and encoders
joblib.dump(model, 'encoders/nba_lineup_model.pkl')
joblib.dump(player_encoder, 'encoders/player_encoder.pkl')
joblib.dump(team_encoder, 'encoders/team_encoder.pkl')
joblib.dump(season_encoder, 'encoders/season_encoder.pkl')

For the parameters we ran multiple tests, basically a lot of trial and error. We used Hyperparameter Tuning but were having issues with the running time, so we manually tested a variety of parameters. We eventually settled on the parameters above **(n_estimators = 300, max_depth=None)** as we were getting the most accurate results with this combination. **EXPLAIN THE PARAMETERS HERE**

## Stage 4: Prediction/Testing
### `generate_test_cases(dataframe, num_of_tests)`
Automatically generates test cases consisting of:
- Home team
- Season
- 4 Home Players
- 5 Away Players
- The 5th Home Player (for comparison with model predictions)

In [ ]:
def generate_test_cases(df, num_test_cases):
    df_filtered = df[df['outcome'] == 1]
    test_cases = []
    for _ in range(num_test_cases):
        random_row = df_filtered.sample(1).iloc[0]
        home_team = random_row['home_team']
        season = random_row['season']
        home_players_4 = sorted([random_row[f'home_{i}'] for i in range(4)])
        away_players_5 = sorted([random_row[f'away_{i}'] for i in range(5)])
        true_fifth_player = random_row['home_4']
        test_cases.append({
            'home_team': home_team,
            'season': season,
            'home_players_4': home_players_4,
            'away_players_5': away_players_5,
            'true_fifth_player': true_fifth_player
        })
    return test_cases

### `evaluate_accuracy(test_cases[], k)`
Passes test cases to `predict_fifth_player`, checking if the actual 5th player is among the model’s top 2 predicted choices. We have the function returning the top 2 players because there can be multiple players in the data that provide a winning outcome for any given 4-player-lineup. This also prints each test case and the model's result while the program runs, so it's easy to see which cases are failing.

In [ ]:
def evaluate_accuracy(test_cases, k):
    top_k_accuracies = []
    for case in test_cases:
        # Return top 3 players for the current test case
        top_k_players = predict_fifth_player(
            case['home_team'],
            case['season'],
            case['home_players_4'],
            case['away_players_5'],
            k
        )
        
        # Calculate if the real player is in the top 3 choices
        if(case['true_fifth_player'] in top_k_players[:k]):
            success = True
        else:
            success = False

        top_k_accuracies.append(success)
        
        # Print results for the current test case
        print(f"Test Case: {case['home_team']} ({case['season']})")
        print(f"Home Players (4): {case['home_players_4']}")
        print(f"Away Players (5): {case['away_players_5']}")
        print(f"True Fifth Player: {case['true_fifth_player']}")
        print(f"Top {k} Predicted Players: {top_k_players}")
        print(f"Success: {success}")
        print("-" * 50)
    
    # Calculate overall accuracy
    overall_accuracy = np.mean(top_k_accuracies)
    print(f"Overall Top-{k} Accuracy: {overall_accuracy:.2f}")

### `predict_fifth_player(home team, season, 4_home_players, 5_away_players, k)`
Predicts the best player to complete the home lineup by:
1. Loading the trained model and encoders.
2. Retrieving the correct roster of eligible players for the given team and season.
3. Encoding the test case.
4. Evaluating all possible lineup combinations using the model.
5. Returning the top 2 players with the highest probabilities.

In [ ]:
def predict_fifth_player(home_team, season, home_players_4, away_players_5, k):
    model = joblib.load('encoders/nba_lineup_model.pkl')
    player_encoder = joblib.load('encoders/player_encoder.pkl')
    team_encoder = joblib.load('encoders/team_encoder.pkl')
    season_encoder = joblib.load('encoders/season_encoder.pkl')

    key = (home_team, season)
    eligible_players = rosters_dict.get(key, set()) - set(home_players_4)
    if not eligible_players:
        return None

    eligible_players = list(eligible_players)
    candidates = []
    for candidate in eligible_players:
        home_lineup = sorted(home_players_4 + [candidate])
        home_encoded = [player_encoder.transform([p])[0] for p in home_lineup]
        away_encoded = [player_encoder.transform([p])[0] for p in away_players_5]
        features = [team_encoder.transform([home_team])[0], season_encoder.transform([season])[0]] + home_encoded + away_encoded
        candidate_df = pd.DataFrame([features], columns=X.columns)
        candidates.append(candidate_df)

    all_candidates = pd.concat(candidates)
    probs = model.predict_proba(all_candidates)[:, 1]
    top_k_indices = np.argsort(probs)[-k:][::-1]
    top_k_players = [eligible_players[i] for i in top_k_indices]
    return top_k_players

## Conclusion
On 500 test cases, our model provided an accuracy of ~. However, 500 cases takes a very long time to run, so the code below runs 50 test cases (will still take a few minutes):

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import os, joblib, numpy as np

# ------------- LOADING & PROCESSING DATA ------------- # 
# Function to load and preprocess the data
def load_data(data_dir):
    all_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
    df_list = []

    for file in all_files:  # Append all csv files into one dataframe to be used later
        df = pd.read_csv(file)
        df_list.append(df)
    df = pd.concat(df_list, ignore_index=True)
    
    # Function to sort all players alphabetically in the dataframe
    def sort_players(row):
        home_players = sorted([row[f'home_{i}'] for i in range(5)])
        away_players = sorted([row[f'away_{i}'] for i in range(5)])
        for i in range(5):
            row[f'home_{i}'] = home_players[i]
            row[f'away_{i}'] = away_players[i]
        return row
    
    # Apply the sorting on the dataframe
    df = df.apply(sort_players, axis=1)
    
    return df

# Function to create roster for each team and season
def create_roster(df):
    # Create a roster for each home team and season. 
    # Ex. GSW 2015 would be one roster, in 2016 it would be a different one.
    roster = df.groupby(['home_team', 'season'])[[f'home_{i}' for i in range(5)]].apply(
        lambda x: set(pd.unique(x.values.ravel()))
    ).to_dict()
    return roster

# Function to encode all categorical features to numerical values 
def encode_features(df, player_encoder, team_encoder, season_encoder):
    df['home_team_encoded'] = team_encoder.transform(df['home_team'])
    df['away_team_encoded'] = team_encoder.transform(df['away_team'])
    df['season_encoded'] = season_encoder.transform(df['season'])
    for i in range(5):
        df[f'home_{i}_encoded'] = player_encoder.transform(df[f'home_{i}'])
        df[f'away_{i}_encoded'] = player_encoder.transform(df[f'away_{i}'])
    return df


# ------------- PREDICTION, TESTING ACCURACY ------------- #
# Prediction function, predicts 3 best options for 5th player to maximize winning.
def predict_fifth_player(home_team, season, home_players_4, away_players_5, k):
    # Load encoders and model
    model = joblib.load('encoders/nba_lineup_model.pkl')
    player_encoder = joblib.load('encoders/player_encoder.pkl')
    team_encoder = joblib.load('encoders/team_encoder.pkl')
    season_encoder = joblib.load('encoders/season_encoder.pkl')
    
    # Get the appropriate roster (eligible players) based on home team and season of input case
    key = (home_team, season)
    eligible_players = rosters_dict.get(key, set())
    eligible_players = eligible_players - set(home_players_4)
    if not eligible_players:
        return None
    eligible_players = list(eligible_players)
    
    # Encode features from input case
    home_team_enc = team_encoder.transform([home_team])[0]
    season_enc = season_encoder.transform([season])[0]
    away_encoded = [player_encoder.transform([p])[0] for p in away_players_5]
    
    # Evaluate all eligible players
    candidates = []
    for candidate in eligible_players:
        # Create and encode home lineup with the current eligible player
        home_lineup = sorted(home_players_4 + [candidate])  # Sorting here to ensure that when the candidate is appended, the lineup stays sorted as in the dataset and trained model
        home_encoded = [player_encoder.transform([p])[0] for p in home_lineup]

        # Features to be saved in the dataframe (encoded versions of all the input case info)
        features = [home_team_enc, season_enc]
        for i in range(5):
            features.extend([home_encoded[i]])
        for i in range(5):
            features.extend([away_encoded[i]])

        # Convert the candidate data into a dataframe (with column names) (this is necessary because the model expects the input as a dataframe)
        candidate_df = pd.DataFrame(
            [features], columns=['home_team_encoded', 'season_encoded', 
                                    'home_0_encoded', 'home_1_encoded', 'home_2_encoded', 'home_3_encoded', 'home_4_encoded',
                                    'away_0_encoded', 'away_1_encoded', 'away_2_encoded', 'away_3_encoded', 'away_4_encoded'])
        candidates.append(candidate_df) # Save each dataframe into the list of candidates
    
    if not candidates:
        return None
    
    # Use model to predict winning probabilities
    all_candidates = pd.concat((candidates)) # Combine each dataframe in the list into one large dataframe
    probs = model.predict_proba(all_candidates)[:, 1] # Model predicts the win probability for each candidate lineup.
    
    # Return the top-2 players that provide the highest winning %
    top_k_indices = np.argsort(probs)[-k:][::-1]
    top_k_players = [eligible_players[i] for i in top_k_indices]
    return top_k_players

# Function to automatically generate test cases
def generate_test_cases(df, num_test_cases):
    # Filter rows where outcome is 1
    df_filtered = df[df['outcome'] == 1]
    
    test_cases = []
    for _ in range(num_test_cases):
        # Randomly select a row from the filtered dataset
        random_row = df_filtered.sample(1).iloc[0]
        
        # Extract the parameters to be used in the model
        home_team = random_row['home_team']
        season = random_row['season']
        home_players_4 = sorted([random_row[f'home_{i}'] for i in range(4)])  # First 4 home players
        away_players_5 = sorted([random_row[f'away_{i}'] for i in range(5)])  # All 5 away players
        
        # Store the actual 5th player (for calculating accuracy)
        true_fifth_player = random_row['home_4']
        
        # Append the test case
        test_cases.append({
            'home_team': home_team,
            'season': season,
            'home_players_4': home_players_4,
            'away_players_5': away_players_5,
            'true_fifth_player': true_fifth_player
        })
    return test_cases

def evaluate_accuracy(test_cases, k):
    top_k_accuracies = []
    for case in test_cases:
        # Return top 2 players for the current test case
        top_k_players = predict_fifth_player(
            case['home_team'],
            case['season'],
            case['home_players_4'],
            case['away_players_5'],
            k
        )
        
        # Calculate if the real player is in the top 2 choices
        if(case['true_fifth_player'] in top_k_players[:k]):
            success = True
        else:
            success = False

        top_k_accuracies.append(success)
        
        # Print results for the current test case
        print(f"Test Case: {case['home_team']} ({case['season']})")
        print(f"Home Players (4): {case['home_players_4']}")
        print(f"Away Players (5): {case['away_players_5']}")
        print(f"True Fifth Player: {case['true_fifth_player']}")
        print(f"Top {k} Predicted Players: {top_k_players}")
        print(f"Success: {success}")
        print("-" * 50)
    
    # Calculate overall accuracy
    overall_accuracy = np.mean(top_k_accuracies)
    print(f"Overall Top-{k} Accuracy: {overall_accuracy:.2f}")


# ------------- MAIN CODE ------------- #
data_dir = './csv_files'
df = load_data(data_dir) # Initialize dataframe

rosters_dict = create_roster(df) # Intialize the dictionary with all eligible players for each team + season

# Initialize encoders
player_encoder = LabelEncoder()
team_encoder = LabelEncoder()
season_encoder = LabelEncoder()

# Fit all encoders on the data
all_players = pd.unique(df[[f'home_{i}' for i in range(5)] + [f'away_{i}' for i in range(5)]].values.ravel()) # Player encoder for all player columns
player_encoder.fit(all_players)
teams = pd.unique(pd.concat([df['home_team'], df['away_team']])) # Team encoder
team_encoder.fit(teams)
seasons = pd.unique(df['season']) # Season encoder
season_encoder.fit(seasons)

# Add encoded (numerical) versions of each column to the dataframe
df = encode_features(df, player_encoder, team_encoder, season_encoder)

    # ------------- TRAINING MODEL ------------- #
# Gather input features and target
X = df[['home_team_encoded', 'season_encoded', 
       'home_0_encoded', 'home_1_encoded', 'home_2_encoded', 'home_3_encoded', 'home_4_encoded',
       'away_0_encoded', 'away_1_encoded', 'away_2_encoded', 'away_3_encoded', 'away_4_encoded']]
y = df['outcome']

# Train Random Forest model
model = RandomForestClassifier(n_estimators=300, max_depth=None, random_state=1, n_jobs=-1)
model.fit(X, y)

# Save model and encoders
joblib.dump(model, 'encoders/nba_lineup_model.pkl')
joblib.dump(player_encoder, 'encoders/player_encoder.pkl')
joblib.dump(team_encoder, 'encoders/team_encoder.pkl')
joblib.dump(season_encoder, 'encoders/season_encoder.pkl')

# Generate test cases
num_test_cases = 50
test_cases = generate_test_cases(df, num_test_cases)

# Evaluate model accuracy (with top-2 predicted players)
k = 2
evaluate_accuracy(test_cases, k)

# RF Parameter Tests:
# (n_estimators, max_depth)
# 200, 10   = 42% -- very fast
# 100, 10   = 50% -- very fast
# 200, 20   = 89% -- relatively slow
# 300, None = 95% -- very slow